# Script to convert BGC-Argo profiles Ed and Lu data in counts into physics units.
NB : This code is adapted to a priliminary version where profiles files are located in aux/coriolis/
and calibration files are from Edouar database.

In [ ]:
import sys
sys.executable

In [ ]:
# Import packages

import numpy as np
import pandas as pd
import xarray as xr
import os
import glob
import cmocean
#import matplotlib

import sys
# add path to the folder which contains Toolbox_RAMSES.py script
sys.path.append('/Users/charlotte.begouen/Documents/Hyperspectral_floats_Herve')
import Toolbox_RAMSES as tools


In [ ]:
# Define filepathes
root = '/Users/charlotte.begouen/Documents/Hyperspectral_floats_Herve/'
# path for metadate folder
meta_dir = root + 'float_profile/'
# path for profiles folder
profile_dir = root + 'float_profile/'
# path to save profiles in physics units
profile_dir_txt = root + 'Outputs/'

In [ ]:
# Choose float WMO that you want to convert
n_float = '6990503' 

In [ ]:
# files names
float_dir = profile_dir+n_float+'/'
meta_name = n_float+'_meta_aux.nc'
meta = xr.open_dataset(float_dir+meta_name)

# find calibration file names associated to this float sensors :
tab = pd.read_table('WMOvsNSerie.txt') 
n_series = tab[ tab.WMO==int(n_float) ]

# open calibration files (retrieved from Edouard)
path_cal = '/Users/charlotte.begouen/Documents/Hyperspectral_floats_Herve/'
path_cal_Ed = path_cal+n_series.N_Serie[ n_series.EdLu=='Ed' ].iloc[0]+'/'
path_cal_Lu = path_cal+n_series.N_Serie[ n_series.EdLu=='Lu' ].iloc[0]+'/'

file_Ed = glob.glob('*AllCal*',root_dir=path_cal_Ed)
file_Lu = glob.glob('*AllCal*',root_dir=path_cal_Lu)

# # open bgc-argo netcdf file
# files_sorted = sorted(os.listdir(float_dir))
# file = xr.open_dataset(float_dir+files_sorted[5])
# file

## Create a csv file with data in physics units

In [29]:
Ed_physic, Lu_physic = pd.DataFrame(), pd.DataFrame()

# list of the float's profiles sorted in alphabetic order
files_name = sorted(os.listdir(float_dir+'profiles'))

for file_name in files_name :
    print(file_name)
    if file_name == '.DS_Store':
        continue

    # open netcdf
    file = xr.open_dataset(float_dir+'profiles'+file_name)

    # Find index in STATION_PARAMETERS values where Radiance and Irradiance are to know what is the N_PROF of Ed and Lu.
    Ed_n_prof = np.where(file.STATION_PARAMETERS.values==b'RAW_DOWNWELLING_IRRADIANCE                                      ')[0][0]
    Lu_n_prof = np.where(file.STATION_PARAMETERS.values==b'RAW_UPWELLING_RADIANCE                                          ')[0][0]

    # format ramses data
    # careful : some float have different pixel binning or pixel stop parameters. 
    #You need to uncomment parameter when it is not the commun config.
    Ed_physic_profile, Lu_physic_profile = tools.format_ramses(float_dir+file_name,meta_dir+meta_name,path_cal_Ed+file_Ed[0],
                                                               path_cal_Lu+file_Lu[0], Ed_n_prof, Lu_n_prof)#, PixelBinning=1)#, PixelStop=144)
    
    # format into a complete dataframe
    profile_Ed = pd.DataFrame({'CRUISE': [file_name[1:8]]*Ed_physic_profile.shape[0],
                               'CYCLE': [int(file_name[-10:-7])]*Ed_physic_profile.shape[0],
                               'WMO': [file_name[1:-7]]*Ed_physic_profile.shape[0],
                               'TIME': [file.JULD.sel(N_PROF=Ed_n_prof).values]*Ed_physic_profile.shape[0],
                               'lon': [file.lonGITUDE.sel(N_PROF=Ed_n_prof).values]*Ed_physic_profile.shape[0],
                               'lat': [file.latITUDE.sel(N_PROF=Ed_n_prof).values]*Ed_physic_profile.shape[0],
                               'PRES_FLOAT' : file.PRES.sel(N_PROF=Ed_n_prof).values[0:Ed_physic_profile.shape[0]] })
    
    profile_Lu = pd.DataFrame({'CRUISE': [file_name[1:8]]*Lu_physic_profile.shape[0],
                               'CYCLE': [int(file_name[-10:-7])]*Lu_physic_profile.shape[0],
                               'WMO': [file_name[1:-7]]*Lu_physic_profile.shape[0],
                                'TIME': [file.JULD.sel(N_PROF=Lu_n_prof).values]*Lu_physic_profile.shape[0],
                               'lon': [file.lonGITUDE.sel(N_PROF=Lu_n_prof).values]*Lu_physic_profile.shape[0],
                               'lat': [file.latITUDE.sel(N_PROF=Lu_n_prof).values]*Lu_physic_profile.shape[0],
                               'PRES_FLOAT' : file.PRES.sel(N_PROF=Lu_n_prof).values[0:Lu_physic_profile.shape[0]] })
    
    Ed_profile = pd.concat([profile_Ed,Ed_physic_profile],axis=1)
    Lu_profile = pd.concat([profile_Lu,Lu_physic_profile],axis=1)
    
    # add to global table of the float
    Ed_physic = pd.concat([Ed_physic,Ed_profile])
    Lu_physic = pd.concat([Lu_physic,Lu_profile])
    
       
# save csv file
Ed_physic.to_csv(profile_dir_txt+file_name[1:8]+'_Ed.csv', index=False)
Lu_physic.to_csv(profile_dir_txt+file_name[1:8]+'_Lu.csv', index=False)
print('All files have been converted into a physics units table')

.DS_Store
R6990503_002_aux.nc


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf', 'scipy']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

## Plot data to check

In [ ]:
import cmocean
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# to save fig
to_save = '/home/lou/Documents/These/phd_axe1/Calibration_RAMSES/Outputs/raw_EdLu_210823/Figures/'

In [ ]:
# import files just created
Ed_physic = pd.read_csv(profile_dir_txt+'6990514_Ed.csv')
Lu_physic = pd.read_csv(profile_dir_txt+'6990514_Lu.csv')

In [ ]:
# choose a profile
Ed_physic_profile = Ed_physic[ Ed_physic.CYCLE==2 ]
Lu_physic_profile = Lu_physic[ Lu_physic.CYCLE==2 ]

In [ ]:
# setup the normalization and the colormap
nValues=Ed_physic_profile.Post_Pres
normalize = mcolors.Normalize(vmin=nValues.min(), vmax=nValues.max())
colormap = plt.get_cmap(cmocean.cm.haline, len(Ed_physic_profile.Post_Pres)).reversed()

# wavelength
wavelength = pd.to_numeric(Ed_physic_profile.columns[8:]).to_list()

# plot figure
plt.figure()
for i in range(len(Ed_physic_profile.Post_Pres)):
    plt.plot(wavelength, Ed_physic_profile.iloc[i,8:], label=Ed_physic_profile.Post_Pres.iloc[i], c=colormap(i), linewidth=0.5)

# setup axis
plt.xlim(300,800)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Ed ($W.nm^{-1}.m^{-2}$)')
plt.yscale('log')
plt.title('{} \n {} \n Lon:{} , Lat:{}'.format(Ed_physic_profile.WMO.iloc[0],Ed_physic_profile.date.iloc[0],round(Ed_physic_profile.lon.iloc[0],2),round(Ed_physic_profile.lat.iloc[0],2)))


# setup the colorbar
scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap )
scalarmappaple.set_array(nValues)
plt.colorbar(scalarmappaple, label='depth (db)')


plt.savefig(to_save+'Edvswave_'+Lu_physic_profile.WMO.iloc[0]+'.png', dpi=300, bbox_inches='tight')

In [ ]:
# setup the normalization and the colormap
nValues=Lu_physic_profile.Post_Pres
normalize = mcolors.Normalize(vmin=nValues.min(), vmax=nValues.max())
colormap = plt.get_cmap(cmocean.cm.haline, len(Lu_physic_profile.Post_Pres)).reversed()

# wavelength
wavelength = pd.to_numeric(Ed_physic_profile.columns[8:]).to_list()

# plot figure
plt.figure()
for i in range(len(Lu_physic_profile.Post_Pres)):
    plt.plot(wavelength, Lu_physic_profile.iloc[i,8:], label=Lu_physic_profile.Post_Pres.iloc[i], c=colormap(i), linewidth=0.5)

# setup axis
plt.xlim(300,800)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Lu ($W.nm^{-1}.m^{-2}.sr^{-1}$)')
plt.yscale('log')
plt.title('{} \n {} \n Lon:{} , Lat:{}'.format(Lu_physic_profile.WMO.iloc[0],Lu_physic_profile.date.iloc[0],round(Lu_physic_profile.lon.iloc[0],2),round(Lu_physic_profile.lat.iloc[0],2)))

# setup the colorbar
scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap )
scalarmappaple.set_array(nValues)
plt.colorbar(scalarmappaple, label='depth (db)')

plt.savefig(to_save+'Luvswave_'+Lu_physic_profile.WMO.iloc[0]+'.png', dpi=300, bbox_inches='tight')

### Test plot en 3D (non fructueux pour l'instant)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# setup the normalization and the colormap
nValues=Lu_physic.Post_Pres
normalize = mcolors.Normalize(vmin=nValues.min(), vmax=nValues.max())
colormap = plt.get_cmap(cmocean.cm.haline, len(Lu_physic_profile.Post_Pres)).reversed()

# make data
wavelength = pd.to_numeric(Ed_physic.columns[7:]).to_list()
time = Ed_physic.date.to_list()
depth = Ed_physic.Post_Pres.to_list()

X, Y, Z = np.meshgrid(wavelength, time, depth)

# Création de la figure et de l'axe 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Tracé du plot 3D
scatter = ax.plot(X.flatten(), Y.flatten(), Z.flatten(),
                     c=Ed_physic_profile.iloc[:,7:].flatten(), cmap=cmocean.cm.thermal, linewidth=0.5)

# for i in range(len(Lu_physic_profile.Post_Pres)):
#     ax.plot_surface(wavelength, time, Lu_physic_profile.iloc[i,7:], label=Lu_physic_profile.Post_Pres.iloc[i], c=colormap(i), linewidth=0.5)

# setup axis
# plt.xlim(300,800)
# plt.xlabel('Wavelength (nm)')
# plt.ylabel('Ed ($W.nm^{-1}.m^{-2}.sr^{-1}$)')
# plt.yscale('log')

# Configuration de la colorbar
cbar = fig.colorbar(scatter, ax=ax)
cbar.set_label('Intensité de la lumière')

# Affichage de la figure
plt.show()